In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import gzip
import time
import shutil
import requests



In [ ]:
def download():
    i = requests.get('http://data.dot.state.mn.us/iris_xml/incident.xml.gz')
    with open('incidents.xml', 'w') as handle:
        handle.write(gzip.decompress(i.content).decode('utf-8'))
    d = requests.get('http://data.dot.state.mn.us/iris_xml/det_sample.xml.gz')
    with open('det_sample.xml', 'w') as handle:
        handle.write(gzip.decompress(d.content).decode('ISO-8859-1'))


In [28]:
def data_check():
        XMLfile = "incident.xml"
        try:
            with open('crash_data.csv', 'r') as CD:
                incidents()
        except FileNotFoundError:
                All_Crash_Data = pd.DataFrame(columns=['Name', 'Date', 'Direction', 'Road', 'Location', 'Event'])
                with open('crash_data.csv', 'w') as f:
                    All_Crash_Data.to_csv(f, header=True)
                    incidents()
        XMLfile = "det_sample.xml"
        try:
            with open('detector_data.csv', 'r') as CD:
                detectors()
        except FileNotFoundError:
                Detector_Data = pd.DataFrame(columns=['Sensor', 'Time', 'Occupancy', 'Speed', 'Flow'])
                with open('detector_data.csv', 'w') as f:
                    Detector_Data.to_csv(f, header=True)
                    detectors()


In [63]:
def incidents():
        XMLfile = "incidents.xml"


        dates = []
        incident_dirs = []
        roads = []
        locations = []
        names = []
        events = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
        for child in root:
            try:
                dates.append(child.attrib['event_date'])
            except KeyError:
                dates.append("NA")
            try:
                names.append(str(child.attrib['name']))
            except KeyError:
                name.append("NA")
            try:
                incident_dirs.append(child.attrib['dir'])
            except KeyError:
                incident_dir.append("NA")
            try:
                roads.append(child.attrib['road'])
            except KeyError:
                roads.append('NA')
            try:
                locations.append(child.attrib['location'])
            except KeyError:
                locations.append("NA")
            try: 
                event = child.attrib['event_type'].split("_", 1)
                events.append(event[1])
            except KeyError:
                events.append("NA")



        DF = pd.DataFrame({"Name" : names,
                           "Date" : dates,
                           "Direction": incident_dirs,
                           "Road" : roads,
                           "Location" : locations,
                           "Event" : events})

        print(DF)

        print("Incident Data Parsed")

        with open('crash_data.csv', 'a') as f:
            DF.to_csv(f, header=False)


In [64]:
def detectors():
        XMLfile = "det_sample.xml"

        sensors = []
        times = []
        flows = []
        occupancies = []
        speeds = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
        for child in root:
            try:
                sensors.append(child.attrib['sensor'])
            except KeyError:
                sensors.append("NA")
#             try:
#                 times.append(str(child.attrib['time_stamp']))
#             except KeyError:
#                 times.append("NA")
            try:
                flows.append(child.attrib['sample flow'])
            except KeyError:
                flows.append("NA")
            try:
                occupancies.append(child.attrib['occ'])
            except KeyError:
                occupancies.append('NA')
            try:
                speeds.append(child.attrib['speed'])
            except KeyError:
                speeds.append("NA")



        DF = pd.DataFrame({"Sensor" : sensors,
#                            "Time" : times,
                           "Occupancy": occupancies,
                           "Speed" : speeds,
                           "Flow" : flows})

        print(DF)
        print("Detector Data Parsed")

        with open('detector_data.csv', 'a') as f:
            DF.to_csv(f, header=False)

In [65]:
def stations():
        XMLfile = "stat_config.xml"

        decription = []
        times = []
        detectors = []
        lats = []
        lngs = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
#         print(root.findall("./time_stamp"))
        for child in root:
            try:
                decription.append(child.attrib['description'])
            except KeyError:
                decription.append("NA")
#             try:
#                 times.append(str(child.attrib['tms_config_time_stamp']))
#             except KeyError:
#                 times.append("NA")
            try:
                detectors.append(child.attrib['name'])
            except KeyError:
                detectors.append("NA")
            try:
                lats.append(child.attrib['lat'])
            except KeyError:
                lats.append('NA')
            try:
                lngs.append(child.attrib['lon'])
            except KeyError:
                lngs.append("NA")



        DF = pd.DataFrame({"Label" : decription,
                           "Sensor" : detectors,
#                            "Time" : times,
                           "Lat": lats,
                           "Lng" : lngs})
        
        DF = DF.dropna(thresh=2)
        print(DF)


        with open('stat_config.csv', 'a') as f:
            DF.to_csv(f, header=False)


In [67]:
s = requests.get('http://data.dot.state.mn.us/iris_xml/metro_config.xml.gz')
with open('stat_config.xml', 'w') as handle:
    handle.write(gzip.decompress(s.content).decode('utf-8'))

XMLfile = "stat_config.xml"
try:
    with open('stat_config.csv', 'r') as CD:
        stations()
except FileNotFoundError:
        Station_Data = pd.DataFrame(columns=['Label', 'Time', 'Detectors', 'Lat', 'Lng'])
        with open('stat_config.csv', 'w') as f:
            Station_Data.to_csv(f, header=True)
            stations()

                             Label     Sensor       Lat        Lng
0                               NA         NA        NA         NA
1                               NA         NA        NA         NA
2                               NA         NA        NA         NA
3                               NA         NA        NA         NA
4                               NA         NA        NA         NA
5                               NA         NA        NA         NA
6                               NA         NA        NA         NA
7                               NA         NA        NA         NA
8                               NA         NA        NA         NA
9                               NA         NA        NA         NA
10                              NA         NA        NA         NA
11                              NA         NA        NA         NA
12                              NA         NA        NA         NA
13                              NA         NA        NA       

In [62]:
while True:
    download()
    print("download complete")
    data_check()
    print("Parsing Complete, sleeping 30s")
    time.sleep(30)

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
2   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
3   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
4   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
5       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
6       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
7   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB      I-35E   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   
12      L004_9942337  Sun

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
2   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
3   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
4   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
5       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
6       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
7   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB      I-35E   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   
12      L004_9942337  Sun

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
2   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
3   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
4   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
5       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
6       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
7   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB      I-35E   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   
12      L004_9942337  Sun

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018093017394538  Sun Sep 30 17:39:45 CDT 2018        NB      I-35W   
2   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
3   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
4   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
5   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
6       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
7       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
8   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB      I-35E   
9       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
10  2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
11  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
12      L004_9942345  Sun

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018093017394538  Sun Sep 30 17:39:45 CDT 2018        NB      I-35W   
2   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
3   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
4   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
5   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
6       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
7       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
8   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB      I-35E   
9       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
10  2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
11  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
12      L004_9942345  Sun

...

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018093017394538  Sun Sep 30 17:39:45 CDT 2018        NB      I-35W   
2   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
3   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
4   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
5   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
6       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
7       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   

           Location     

                             Label     Sensor       Lat        Lng
0                               NA         NA        NA         NA
1                               NA         NA        NA         NA
2                               NA         NA        NA         NA
3                               NA         NA        NA         NA
4                               NA         NA        NA         NA
5                               NA         NA        NA         NA
6                               NA         NA        NA         NA
7                               NA         NA        NA         NA
8                               NA         NA        NA         NA
9                               NA         NA        NA         NA
10                              NA         NA        NA         NA
11                              NA         NA        NA         NA
12                              NA         NA        NA         NA
13                              NA         NA        NA       

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018093017394538  Sun Sep 30 17:39:45 CDT 2018        NB      I-35W   
2   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
3   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
4   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
5   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
6       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
7       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   

           Location     

                Name                          Date Direction       Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB     T.H.62   
1   2018093017394538  Sun Sep 30 17:39:45 CDT 2018        NB      I-35W   
2   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB    U.S.169   
3   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB    U.S.169   
4   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB    T.H.100   
5   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB      I-35W   
6       L004_9942351  Sun Sep 30 17:36:42 CDT 2018        EB     T.H.62   
7       L004_9942347  Sun Sep 30 17:34:02 CDT 2018        EB      I-394   
8       L004_9942348  Sun Sep 30 17:34:32 CDT 2018        WB  I-394 Rev   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB     T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB    T.H.100   
11      L004_9942345  Sun Sep 30 17:33:02 CDT 2018        EB       I-94   

           Location     

KeyboardInterrupt: 